# Librerias

In [70]:
import requests
import pandas as pd
from datetime import datetime
import time
import os
import base64
import json
import numpy as np

# Extraer

In [2]:
url = 'https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0'
response = requests.get(url)
data = response.json()

In [3]:
save_directory = './pokemon-sprites/'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [7]:
list_pkmn_primary_data = []
list_abilities = []
list_game_appears = []
list_stats = []
list_types = []
list_pokedex_description = []
list_evolutions = []

for pokemon in data['results']:
    response_info_pokemon = requests.get(pokemon['url'])
    data_info_pokemon = response_info_pokemon.json()
    
    # Datos principales
    id_pokemon = data_info_pokemon['id']
    pokemon_name = pokemon['name']
    pokemon_order = data_info_pokemon['order']
    pokemon_weight = data_info_pokemon['weight']
    pokemon_height = data_info_pokemon['height']
    
    # Datos de habilidades
    pokemon_abilities = data_info_pokemon['abilities']
    for ability in pokemon_abilities:
        ability_name = ability['ability']['name']
        ability_is_hidden = ability['is_hidden']
        # URL de la habildiad para obtener la descripción
        ability_url = ability['ability']['url']
        response_ability_pokemon = requests.get(ability_url)
        data_ability_pokemon = response_ability_pokemon.json()
        id_ability = data_ability_pokemon['id']
        # Obtener la descripcion en ingles de la la habildiad
        for ability in data_ability_pokemon['flavor_text_entries']:
            if ability['language']['name'] == 'en':
                ability_description = ability['flavor_text']
        
        list_abilities.append([id_pokemon, id_ability, ability_name, ability_is_hidden, ability_description])
    
    # Datos Especie
    species_url = data_info_pokemon['species']['url']
    response_species = requests.get(species_url)
    data_species = response_species.json()
    flavor_text_entries = data_species['flavor_text_entries']
    for entry in flavor_text_entries:
        if entry['language']['name'] == 'en':
            flavor_text = entry['flavor_text']
            flavor_text = flavor_text.replace("\n", ' ')
            game_name = entry['version']['name']
        list_pokedex_description.append([id_pokemon, flavor_text, game_name])

    # Datos Evolucion
    evolution_chain_url = data_species['evolution_chain']['url']
    response_evolution_chain = requests.get(evolution_chain_url)
    data_evolution_chain = response_evolution_chain.json()
    if len(data_evolution_chain['chain']['evolves_to']) > 0:
        for evolve_to_1 in data_evolution_chain['chain']['evolves_to']: 
            evolve_to_pkmn = evolve_to_1['species']['name']
            if len(evolve_to_1['evolution_details']) > 0:
                evol_req_gender = evolve_to_1['evolution_details'][0]['gender']
                evol_req_held_item = evolve_to_1['evolution_details'][0]['held_item']
                evol_req_item = evolve_to_1['evolution_details'][0]['item']
                evol_req_known_move = evolve_to_1['evolution_details'][0]['known_move']
                evol_req_known_move_type = evolve_to_1['evolution_details'][0]['known_move_type']
                evol_req_location = evolve_to_1['evolution_details'][0]['location']
                evol_req_min_affection = evolve_to_1['evolution_details'][0]['min_affection']
                evol_req_min_beauty = evolve_to_1['evolution_details'][0]['min_beauty']
                evol_req_min_happiness = evolve_to_1['evolution_details'][0]['min_happiness']
                evol_req_min_level = evolve_to_1['evolution_details'][0]['min_level']
                evol_req_needs_overworld_rain = evolve_to_1['evolution_details'][0]['needs_overworld_rain']
                evol_req_party_species = evolve_to_1['evolution_details'][0]['party_species']
                evol_req_party_type = evolve_to_1['evolution_details'][0]['party_type']
                evol_req_relative_physical_stats = evolve_to_1['evolution_details'][0]['relative_physical_stats']
                evol_req_time_of_day = evolve_to_1['evolution_details'][0]['time_of_day']
                evol_req_trade_species = evolve_to_1['evolution_details'][0]['trade_species']
                evol_req_turn_upside_down = evolve_to_1['evolution_details'][0]['turn_upside_down']
            else:
                evol_req_gender = ''
                evol_req_held_item = ''
                evol_req_item = ''
                evol_req_known_move = ''
                evol_req_known_move_type = ''
                evol_req_location = ''
                evol_req_min_affection = ''
                evol_req_min_beauty = ''
                evol_req_min_happiness = ''
                evol_req_min_level = ''
                evol_req_needs_overworld_rain = ''
                evol_req_party_species = ''
                evol_req_party_type = ''
                evol_req_relative_physical_stats = ''
                evol_req_time_of_day = ''
                evol_req_trade_species = ''
                evol_req_turn_upside_down = ''
            list_evolutions.append([id_pokemon,
                evolve_to_pkmn, evol_req_gender, evol_req_held_item
                , evol_req_item, evol_req_known_move, evol_req_known_move_type
                , evol_req_location, evol_req_min_affection, evol_req_min_beauty
                , evol_req_min_happiness, evol_req_min_level, evol_req_needs_overworld_rain
                , evol_req_party_species, evol_req_party_type, evol_req_relative_physical_stats
                , evol_req_time_of_day, evol_req_trade_species, evol_req_turn_upside_down,
            ])
            if len(evolve_to_1['evolves_to']) > 0:
                for evolve_to_2 in evolve_to_1['evolves_to']:
                    evolve_to_pkmn = evolve_to_2['species']['name']
                    if len(evolve_to_2['evolution_details']) > 0:
                        evol_req_gender = evolve_to_2['evolution_details'][0]['gender']
                        evol_req_held_item = evolve_to_2['evolution_details'][0]['held_item']
                        evol_req_item = evolve_to_2['evolution_details'][0]['item']
                        evol_req_known_move = evolve_to_2['evolution_details'][0]['known_move']
                        evol_req_known_move_type = evolve_to_2['evolution_details'][0]['known_move_type']
                        evol_req_location = evolve_to_2['evolution_details'][0]['location']
                        evol_req_min_affection = evolve_to_2['evolution_details'][0]['min_affection']
                        evol_req_min_beauty = evolve_to_2['evolution_details'][0]['min_beauty']
                        evol_req_min_happiness = evolve_to_2['evolution_details'][0]['min_happiness']
                        evol_req_min_level = evolve_to_2['evolution_details'][0]['min_level']
                        evol_req_needs_overworld_rain = evolve_to_2['evolution_details'][0]['needs_overworld_rain']
                        evol_req_party_species = evolve_to_2['evolution_details'][0]['party_species']
                        evol_req_party_type = evolve_to_2['evolution_details'][0]['party_type']
                        evol_req_relative_physical_stats = evolve_to_2['evolution_details'][0]['relative_physical_stats']
                        evol_req_time_of_day = evolve_to_2['evolution_details'][0]['time_of_day']
                        evol_req_trade_species = evolve_to_2['evolution_details'][0]['trade_species']
                        evol_req_turn_upside_down = evolve_to_2['evolution_details'][0]['turn_upside_down']
                    else:
                        evol_req_gender = ''
                        evol_req_held_item = ''
                        evol_req_item = ''
                        evol_req_known_move = ''
                        evol_req_known_move_type = ''
                        evol_req_location = ''
                        evol_req_min_affection = ''
                        evol_req_min_beauty = ''
                        evol_req_min_happiness = ''
                        evol_req_min_level = ''
                        evol_req_needs_overworld_rain = ''
                        evol_req_party_species = ''
                        evol_req_party_type = ''
                        evol_req_relative_physical_stats = ''
                        evol_req_time_of_day = ''
                        evol_req_trade_species = ''
                        evol_req_turn_upside_down = ''
                    list_evolutions.append([id_pokemon,
                        evolve_to_pkmn, evol_req_gender, evol_req_held_item
                        , evol_req_item, evol_req_known_move, evol_req_known_move_type
                        , evol_req_location, evol_req_min_affection, evol_req_min_beauty
                        , evol_req_min_happiness, evol_req_min_level, evol_req_needs_overworld_rain
                        , evol_req_party_species, evol_req_party_type, evol_req_relative_physical_stats
                        , evol_req_time_of_day, evol_req_trade_species, evol_req_turn_upside_down,
                    ])

    # Datos de apariciones en versiones
    pokemon_game_indices = data_info_pokemon['game_indices']
    for game in pokemon_game_indices:
        game_name = game['version']['name']
        list_game_appears.append([id_pokemon, game_name])

    # Datos de estadistica
    pokemon_stats = data_info_pokemon['stats']
    for stat in pokemon_stats:
        stat_name = stat['stat']['name']
        stat_base = stat['base_stat']
        stat_effort = stat['effort']
        list_stats.append([id_pokemon, stat_name, stat_base, stat_effort])
            
    # Datos de tipo
    pokemon_types = data_info_pokemon['types']
    for type in pokemon_types:
        type_name = type['type']['name']
        type_number = type['slot']
        list_types.append([id_pokemon, type_name, type_number])

    # Descargar el sprite por defecto del pokemon
    front_default_url = data_info_pokemon['sprites']['front_default']
    if front_default_url:
        image_name = f'{id_pokemon}-{pokemon_name}.png'
        image_path = os.path.join(save_directory, image_name)
        img_response = requests.get(front_default_url)

        if img_response.status_code == 200:
            with open(image_path, 'wb') as img_file:
                img_file.write(img_response.content)
                list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
                print("Registro exitoso: " + pokemon_name)
        else:
            print('Registro exitoso ' + pokemon_name + ' pero falló la descarga del sprite')
    else:
        print('Registro exitoso ' + pokemon_name + ' pero no existe sprite')
        image_path = ''
        list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
    
    time.sleep(0.1)
    

Registro exitoso: bulbasaur
Registro exitoso: ivysaur
Registro exitoso: venusaur
Registro exitoso: charmander
Registro exitoso: charmeleon
Registro exitoso: charizard
Registro exitoso: squirtle
Registro exitoso: wartortle
Registro exitoso: blastoise
Registro exitoso: caterpie
Registro exitoso: metapod
Registro exitoso: butterfree
Registro exitoso: weedle
Registro exitoso: kakuna
Registro exitoso: beedrill
Registro exitoso: pidgey
Registro exitoso: pidgeotto
Registro exitoso: pidgeot
Registro exitoso: rattata
Registro exitoso: raticate
Registro exitoso: spearow
Registro exitoso: fearow
Registro exitoso: ekans
Registro exitoso: arbok
Registro exitoso: pikachu
Registro exitoso: raichu
Registro exitoso: sandshrew
Registro exitoso: sandslash
Registro exitoso: nidoran-f
Registro exitoso: nidorina
Registro exitoso: nidoqueen
Registro exitoso: nidoran-m
Registro exitoso: nidorino
Registro exitoso: nidoking
Registro exitoso: clefairy
Registro exitoso: clefable
Registro exitoso: vulpix
Registro 

In [8]:
df_pkmn_primary_data_extraer = pd.DataFrame(
    list_pkmn_primary_data,
    columns = ['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']
)

df_pkmn_abilities_extraer = pd.DataFrame(
    list_abilities,
    columns = ['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']
)

df_pkmn_game_appears_extraer = pd.DataFrame(
    list_game_appears,
    columns = ['pkmn_id', 'game_version']
)

df_pkmn_stats_extraer = pd.DataFrame(
    list_stats,
    columns = ['pkmn_id', 'name', 'stat', 'effort']
)

df_pkmn_types_extraer = pd.DataFrame(
    list_types,
    columns = ['pkmn_id', 'name', 'type_number']
)

df_pkmn_pokedex_description_extraer = pd.DataFrame(
    list_pokedex_description,
    columns = ['pkmn_id', 'flavor_text', 'game_name']
)

df_pkmn_evolution_extraer = pd.DataFrame(
    list_evolutions,
    columns = [
        'pkmn_id', 'evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item'
        , 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type'
        , 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty'
        , 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain'
        , 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats'
        , 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down',
        ]
)

In [10]:
df_pkmn_evolution_extraer.head(100)

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down
0,1,ivysaur,None,None,None,None,None,None,None,None,None,16,False,None,None,None,,None,False
1,1,venusaur,None,None,None,None,None,None,None,None,None,32,False,None,None,None,,None,False
2,2,ivysaur,None,None,None,None,None,None,None,None,None,16,False,None,None,None,,None,False
3,2,venusaur,None,None,None,None,None,None,None,None,None,32,False,None,None,None,,None,False
4,3,ivysaur,None,None,None,None,None,None,None,None,None,16,False,None,None,None,,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,56,primeape,None,None,None,None,None,None,None,None,None,28,False,None,None,None,,None,False
96,56,annihilape,None,None,None,None,None,None,None,None,None,None,False,None,None,None,,None,False
97,57,primeape,None,None,None,None,None,None,None,None,None,28,False,None,None,None,,None,False
98,57,annihilape,None,None,None,None,None,None,None,None,None,None,False,None,None,None,,None,False


In [11]:
df_pkmn_primary_data_extraer['extraer_datetime'] = datetime.now()
df_pkmn_abilities_extraer['extraer_datetime'] = datetime.now()
df_pkmn_game_appears_extraer['extraer_datetime'] = datetime.now()
df_pkmn_stats_extraer['extraer_datetime'] = datetime.now()
df_pkmn_types_extraer['extraer_datetime'] = datetime.now()
df_pkmn_pokedex_description_extraer['extraer_datetime'] = datetime.now()
df_pkmn_evolution_extraer['extraer_datetime'] = datetime.now()

In [12]:
df_pkmn_primary_data_extraer.to_csv('fuente/extraer/pkmn_primary_data.csv', index = False, sep = '|')
df_pkmn_abilities_extraer.to_csv('fuente/extraer/pkmn_abilities.csv', index = False, sep = '|')
df_pkmn_game_appears_extraer.to_csv('fuente/extraer/pkmn_game_appears.csv', index = False, sep = '|')
df_pkmn_stats_extraer.to_csv('fuente/extraer/pkmn_stats.csv', index = False, sep = '|')
df_pkmn_types_extraer.to_csv('fuente/extraer/pkmn_types.csv', index = False, sep = '|')
df_pkmn_pokedex_description_extraer.to_csv('fuente/extraer/pkmn_pokedex_description.csv', index = False, sep= '|')
df_pkmn_evolution_extraer.to_csv('fuente/extraer/pkmn_evolution.csv', index= False, sep='|')

# Transformar

In [103]:
df_pkmn_data_transformar = pd.read_csv('fuente/extraer/pkmn_primary_data.csv', delimiter = '|')
df_pkmn_abilities_transformar = pd.read_csv('fuente/extraer/pkmn_abilities.csv', delimiter = '|')
df_pkmn_types_transformar = pd.read_csv('fuente/extraer/pkmn_types.csv', delimiter= '|')
df_pkmn_game_appears_transformar = pd.read_csv('fuente/extraer/pkmn_game_appears.csv', delimiter='|')
df_pkmn_stats_transformar = pd.read_csv('fuente/extraer/pkmn_stats.csv', delimiter='|')
df_pkmn_pokedex_description_transformar = pd.read_csv('fuente/extraer/pkmn_pokedex_description.csv', delimiter = '|')
df_pkmn_evolution_transformar = pd.read_csv('fuente/extraer/pkmn_evolution.csv', delimiter='|')

In [104]:
df_pkmn_evolution_transformar.head()

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,extraer_datetime
0,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,2024-08-09 10:49:15.051754
1,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,2024-08-09 10:49:15.051754
2,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,2024-08-09 10:49:15.051754
3,2,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,2024-08-09 10:49:15.051754
4,3,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,2024-08-09 10:49:15.051754


In [105]:
# Pkmn Data
df_pkmn_data_transformar = df_pkmn_data_transformar[['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']]
# Pkmn abilities
df_pkmn_abilities_transformar = df_pkmn_abilities_transformar[['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']]
# Pkmn Types
df_pkmn_types_transformar = df_pkmn_types_transformar[['pkmn_id', 'name', 'type_number']]
# Pkmn Gamer Appears
df_pkmn_game_appears_transformar = df_pkmn_game_appears_transformar[['pkmn_id', 'game_version']]
# Pkmn Stats
df_pkmn_stats_transformar = df_pkmn_stats_transformar[['pkmn_id', 'name', 'stat', 'effort']]
# Pkmn Pokedex Description
df_pkmn_pokedex_description_transformar['id_pokedex_description'] = range(1, len(df_pkmn_pokedex_description_transformar) + 1)
df_pkmn_pokedex_description_transformar = df_pkmn_pokedex_description_transformar[['id_pokedex_description', 'pkmn_id', 'game_name', 'flavor_text']]
# Pkmn evolution
df_pkmn_evolution_transformar = df_pkmn_evolution_transformar[['evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item', 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type', 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty', 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain', 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats', 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down']]


## Limpiar tabla evolution

### Limpiar duplicados y agregar indice

In [106]:
df_pkmn_evolution_transformar = df_pkmn_evolution_transformar.drop_duplicates()
df_pkmn_evolution_transformar['id_pkmn_evolution'] = range(1, len(df_pkmn_evolution_transformar)+ 1)
df_pkmn_evolution_transformar = df_pkmn_evolution_transformar[['id_pkmn_evolution','evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item', 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type', 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty', 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain', 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats', 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down']]

### Analizar cada campo de la tabla evolution

#### Campo evol_req_gender
- Los valores 1 representan al requerimiento de que el pokemon debe ser de género hembra
- Los valores 2 representan al requerimiento de que el pokemon debe ser de género macho

In [107]:
df_pkmn_evolution_transformar.groupby('evol_req_gender').size().reset_index(name='count')

,evol_req_gender,count
0,1.0,4
1,2.0,2


In [34]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_gender'] == 1]

,id_pkmn_evolution,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,...,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,NaN,2.0,1.0
552,220,froslass,1.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1
621,243,wormadam,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1
627,245,vespiquen,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1
1051,393,salazzle,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1


In [35]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_gender'] == 2]

,id_pkmn_evolution,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,...,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,NaN,2.0,1.0
463,182,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1
622,244,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,1,1,1


#### Campo evol_req_held_item

In [108]:
df_pkmn_evolution_transformar.groupby('evol_req_held_item').size().reset_index(name='count')

,evol_req_held_item,count
0,"{'name': 'deep-sea-scale', 'url': 'https://pok...",1
1,"{'name': 'deep-sea-tooth', 'url': 'https://pok...",1
2,"{'name': 'dragon-scale', 'url': 'https://pokea...",1
3,"{'name': 'dubious-disc', 'url': 'https://pokea...",1
4,"{'name': 'electirizer', 'url': 'https://pokeap...",1
5,"{'name': 'kings-rock', 'url': 'https://pokeapi...",2
6,"{'name': 'magmarizer', 'url': 'https://pokeapi...",1
7,"{'name': 'metal-coat', 'url': 'https://pokeapi...",2
8,"{'name': 'oval-stone', 'url': 'https://pokeapi...",1
9,"{'name': 'protector', 'url': 'https://pokeapi....",1


In [82]:
def extract_name_from_json(val):
    if pd.notna(val):  # Verifica si el valor no es NaN
        try:
            # Convierte la cadena JSON a un diccionario si es necesario
            json_obj = json.loads(val.replace("'", "\""))
            # Extrae el valor de la llave 'name'
            return json_obj.get('name', np.nan)
        except (json.JSONDecodeError, TypeError):
            # Si falla la conversión, devuelve NaN
            return np.nan
    return np.nan  # Devuelve NaN si el valor es NaN

In [83]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_held_item'] = df_pkmn_evolution_transformar['evol_req_held_item'].apply(extract_name_from_json)

In [ ]:
df_pkmn_evolution_transformar.groupby('evol_req_held_item').size().reset_index(name='count')

#### Campo evol_req_item

In [109]:
df_pkmn_evolution_transformar.groupby('evol_req_item').size().reset_index(name='count')

,evol_req_item,count
0,"{'name': 'auspicious-armor', 'url': 'https://p...",1
1,"{'name': 'black-augurite', 'url': 'https://pok...",1
2,"{'name': 'cracked-pot', 'url': 'https://pokeap...",1
3,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",2
4,"{'name': 'dusk-stone', 'url': 'https://pokeapi...",4
5,"{'name': 'fire-stone', 'url': 'https://pokeapi...",5
6,"{'name': 'ice-stone', 'url': 'https://pokeapi....",1
7,"{'name': 'leaf-stone', 'url': 'https://pokeapi...",5
8,"{'name': 'malicious-armor', 'url': 'https://po...",1
9,"{'name': 'moon-stone', 'url': 'https://pokeapi...",6


#### Campo evol_req_known_move

In [112]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move').size().reset_index(name='count')

,evol_req_known_move,count
0,"{'name': 'ancient-power', 'url': 'https://poke...",3
1,"{'name': 'barb-barrage', 'url': 'https://pokea...",1
2,"{'name': 'double-hit', 'url': 'https://pokeapi...",1
3,"{'name': 'dragon-pulse', 'url': 'https://pokea...",1
4,"{'name': 'hyper-drill', 'url': 'https://pokeap...",1
5,"{'name': 'mimic', 'url': 'https://pokeapi.co/a...",2
6,"{'name': 'psyshield-bash', 'url': 'https://pok...",1
7,"{'name': 'rollout', 'url': 'https://pokeapi.co...",1
8,"{'name': 'stomp', 'url': 'https://pokeapi.co/a...",1
9,"{'name': 'taunt', 'url': 'https://pokeapi.co/a...",1


#### Campo evol_req_known_move_type

In [113]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move_type').size().reset_index(name='count')

,evol_req_known_move_type,count
0,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",1


#### Campo evol_req_location

In [114]:
df_pkmn_evolution_transformar.groupby('evol_req_location').size().reset_index(name='count')

,evol_req_location,count
0,"{'name': 'eterna-forest', 'url': 'https://poke...",1
1,"{'name': 'mount-lanakila', 'url': 'https://pok...",1
2,"{'name': 'mt-coronet', 'url': 'https://pokeapi...",2
3,"{'name': 'sinnoh-route-217', 'url': 'https://p...",1


#### Campo evol_req_min_affection

In [115]:
df_pkmn_evolution_transformar.groupby('evol_req_min_affection').size().reset_index(name='count')

,evol_req_min_affection,count
0,2.0,1


#### Campo evol_req_min_beauty

In [124]:
df_pkmn_evolution_transformar.groupby('evol_req_min_beauty').size().reset_index(name='count')

,evol_req_min_beauty,count
0,171.0,1


#### Campo evol_req_min_happiness

In [125]:
df_pkmn_evolution_transformar.groupby('evol_req_min_happiness').size().reset_index(name='count')

,evol_req_min_happiness,count
0,160.0,15
1,220.0,3


#### Campo evol_req_need_overworld_rain

In [126]:
df_pkmn_evolution_transformar.groupby('evol_req_needs_overworld_rain').size().reset_index(name='count')

,evol_req_needs_overworld_rain,count
0,False,478
1,True,1


#### Campo evol_req_party_species

In [127]:
df_pkmn_evolution_transformar.groupby('evol_req_party_species').size().reset_index(name='count')

,evol_req_party_species,count
0,"{'name': 'remoraid', 'url': 'https://pokeapi.c...",1


#### Campo evol_req_party_type

In [128]:
df_pkmn_evolution_transformar.groupby('evol_req_party_type').size().reset_index(name='count')

,evol_req_party_type,count
0,"{'name': 'dark', 'url': 'https://pokeapi.co/ap...",1


#### Campo evol_req_relative_physical_stats

In [129]:
df_pkmn_evolution_transformar.groupby('evol_req_relative_physical_stats').size().reset_index(name='count')

,evol_req_relative_physical_stats,count
0,-1.0,1
1,0.0,1
2,1.0,1


#### Campo evol_req_time_of_day

In [130]:
df_pkmn_evolution_transformar.groupby('evol_req_time_of_day').size().reset_index(name='count')

,evol_req_time_of_day,count
0,day,9
1,full-moon,1
2,night,8


#### Campo evol_req_trade_species

In [131]:
df_pkmn_evolution_transformar.groupby('evol_req_trade_species').size().reset_index(name='count')

,evol_req_trade_species,count
0,"{'name': 'karrablast', 'url': 'https://pokeapi...",1
1,"{'name': 'shelmet', 'url': 'https://pokeapi.co...",1


#### Campo evol_req_turn_upside_down

In [132]:
df_pkmn_evolution_transformar.groupby('evol_req_turn_upside_down').size().reset_index(name='count')

,evol_req_turn_upside_down,count
0,False,478
1,True,1


In [133]:
df_pkmn_evolution_transformar.head()

,id_pkmn_evolution,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down
0,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False
1,2,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False
6,3,charmeleon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False
7,4,charizard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,False,NaN,NaN,NaN,NaN,NaN,False
12,5,wartortle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False


In [184]:
# Generar una tabla de habilidades únicas
df_abilities_transformar = df_pkmn_abilities_transformar[['ability_id', 'ability_name', 'ability_description']].drop_duplicates().reset_index(drop=True)

In [185]:
# Crear una tabla de relaciones entre Pokémon y habilidades
df_pkmn_abilities_rel_transformar = df_pkmn_abilities_transformar.copy()
df_pkmn_abilities_rel_transformar = df_pkmn_abilities_rel_transformar[['pkmn_id', 'ability_id', 'ability_is_hidden']]

In [186]:
# Agregar información de la imagen en base64 
save_directory
data_imagenes = []
for image_name in os.listdir(save_directory):
    image_path = os.path.join(save_directory, image_name)

    with open(image_path, 'rb') as img_file:
        base64_string = base64.b64encode(img_file.read()).decode('utf-8')
        # img_html = f'<img src="data:image/png;base64,{base64_string}" width="{300}px" height="{300}px"/>'
        img_html = f'<img src="data:image/png;base64,{base64_string}" style="width:{300}px; height:{300}px;" alt="Image"/>'

    data_imagenes.append({
        'pkmn_id': image_name.split('-')[0],
        'image_name': image_name,
        'image_base64': base64_string,
        'img_html': img_html
    })

df_pkmn_images_transformar = pd.DataFrame(data_imagenes)

In [187]:
df_pkmn_data_transformar['transformar_datetime'] = datetime.now()
df_abilities_transformar['transformar_datetime'] = datetime.now()
df_pkmn_abilities_rel_transformar['transformar_datetime'] = datetime.now()
df_pkmn_types_transformar['transformar_datetime'] = datetime.now()
df_pkmn_game_appears_transformar['transformar_datetime'] = datetime.now()
df_pkmn_stats_transformar['transformar_datetime'] = datetime.now()
df_pkmn_images_transformar['transformar_datetime'] = datetime.now()

In [188]:
df_pkmn_data_transformar.to_csv('fuente/transformar/pkmn_primary_data.csv.csv', sep = '|', index = False)
df_abilities_transformar.to_csv('fuente/transformar/pkmn_abilities.csv.csv', sep = '|', index = False)
df_pkmn_abilities_rel_transformar.to_csv('fuente/transformar/pkmn_abitilies_rel.csv', sep = '|', index = False)
df_pkmn_types_transformar.to_csv('fuente/transformar/pkmn_types.csv', sep = '|', index = False)
df_pkmn_game_appears_transformar.to_csv('fuente/transformar/pkmn_game_appears.csv', sep = '|', index = False)
df_pkmn_stats_transformar.to_csv('fuente/transformar/pkmn_stats.csv', sep = '|', index = False)
df_pkmn_images_transformar.to_csv('fuente/transformar/pkmn_images.csv', sep = '|', index = False)

# Cargar

In [189]:
df_pkmn_data_cargar = pd.read_csv('fuente/transformar/pkmn_primary_data.csv.csv', delimiter = '|')
df_abilities_cargar = pd.read_csv('fuente/transformar/pkmn_abilities.csv.csv', delimiter = '|')
df_pkmn_abilities_rel_cargar = pd.read_csv('fuente/transformar/pkmn_abitilies_rel.csv', delimiter = '|')
df_pkmn_types_cargar = pd.read_csv('fuente/transformar/pkmn_types.csv', delimiter = '|')
df_pkmn_game_appears_cargar = pd.read_csv('fuente/transformar/pkmn_game_appears.csv', delimiter = '|')
df_pkmn_stats_cargar = pd.read_csv('fuente/transformar/pkmn_stats.csv', delimiter = '|')
df_pkmn_images_cargar = pd.read_csv('fuente/transformar/pkmn_images.csv', delimiter = '|')

In [190]:
df_pkmn_data_cargar = df_pkmn_data_cargar[['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']]
df_abilities_cargar = df_abilities_cargar[['ability_id', 'ability_name', 'ability_description']]
df_pkmn_abilities_rel_cargar = df_pkmn_abilities_rel_cargar[['pkmn_id', 'ability_id', 'ability_is_hidden']]
df_pkmn_types_cargar = df_pkmn_types_cargar[['pkmn_id', 'name', 'type_number']]
df_pkmn_game_appears_cargar = df_pkmn_game_appears_cargar[['pkmn_id', 'game_version']]
df_pkmn_stats_cargar = df_pkmn_stats_cargar[['pkmn_id', 'name', 'stat', 'effort']]
df_pkmn_images_cargar = df_pkmn_images_cargar[['pkmn_id', 'image_name', 'image_base64', 'img_html']]

In [191]:
df_pkmn_data_cargar['cargar_datetime'] = datetime.now()
df_abilities_cargar['cargar_datetime'] = datetime.now()
df_pkmn_abilities_rel_cargar['cargar_datetime'] = datetime.now()
df_pkmn_types_cargar['cargar_datetime'] = datetime.now()
df_pkmn_game_appears_cargar['cargar_datetime'] = datetime.now()
df_pkmn_stats_cargar['cargar_datetime'] = datetime.now()
df_pkmn_images_cargar['cargar_datetime'] = datetime.now()

In [192]:
df_pkmn_data_cargar.to_csv('fuente/cargar/pkmn_primary_data.csv', sep = '|', index = False)
df_abilities_cargar.to_csv('fuente/cargar/pkmn_abilities.csv', sep = '|', index = False)
df_pkmn_abilities_rel_cargar.to_csv('fuente/cargar/pkmn_abitilies_rel.csv', sep = '|', index = False)
df_pkmn_types_cargar.to_csv('fuente/cargar/pkmn_types.csv', sep = '|', index = False)
df_pkmn_game_appears_cargar.to_csv('fuente/cargar/pkmn_game_appears.csv', sep = '|', index = False)
df_pkmn_stats_cargar.to_csv('fuente/cargar/pkmn_stats.csv', sep = '|', index = False)
df_pkmn_images_cargar.to_csv('fuente/cargar/pkmn_images.csv', sep = '|', index = False)